In [54]:
import tensorflow as tf
import cv2
from cvzone.HandTrackingModule import HandDetector
# from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tensorflow.keras

In [55]:
class Classifier:

    def __init__(self, modelPath, labelsPath=None):
        self.model_path = modelPath
        # Disable scientific notation for clarity
        np.set_printoptions(suppress=True)
        # Load the model
        self.model = tensorflow.keras.models.load_model(self.model_path)

        # Create the array of the right shape to feed into the keras model
        # The 'length' or number of images you can put into the array is
        # determined by the first position in the shape tuple, in this case 1.
        self.data = np.ndarray(shape=(1, 256, 256, 3), dtype=np.float32)
        self.labels_path = labelsPath
        if self.labels_path:
            label_file = open(self.labels_path, "r")
            self.list_labels = []
            for line in label_file:
                stripped_line = line.strip()
                self.list_labels.append(stripped_line)
            label_file.close()
        else:
            print("No Labels Found")

    def getPrediction(self, img, draw= True, pos=(50, 50), scale=2, color = (0,255,0)):
        # resize the image to a 224x224 with the same strategy as in TM2:
#         imgS = cv2.resize(img, (256, 256))
        # turn the image into a numpy array
#         image_array = np.asarray(imgS)
        # Normalize the image
#         normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

        # Load the image into the array
        self.data[0] = img

        # run the inference
        prediction = self.model.predict(self.data)
        indexVal = np.argmax(prediction)

        if draw and self.labels_path:
            cv2.putText(img, str(self.list_labels[indexVal]),
                        pos, cv2.FONT_HERSHEY_COMPLEX, scale, color, 2)

        return list(prediction[0]), indexVal

In [ ]:
cap = cv2.VideoCapture(1)
detector = HandDetector(maxHands=1)

classifier = Classifier("model/imageclassifier.h5", "model/labels3.txt")
# model = tensorflow.keras.models.load_model("model/imageclassifier.h5")

# list_labels = []
# labels_path = labelsPath
# if labels_path:
#     label_file = open(labels_path, "r")
#     for line in label_file:
#         stripped_line = line.strip()
#         list_labels.append(stripped_line)
#     label_file.close()
# else:
#     print("No Labels Found")

offset = 20
imgSize = 256

folder = "letras"
counter = 0

labels = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        imgCropShape = imgCrop.shape

        aspectRatio = h / w

        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            resize = tf.image.resize(imgWhite, (256,256))
#             image = np.array(imgWhite)
            prediction, index = classifier.getPrediction(resize, draw=False)
            print(prediction, index)

        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            resize = tf.image.resize(imgWhite, (256,256))
#             image = np.array(imgWhite)
            prediction, index = classifier.getPrediction(resize, draw=False)

        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)


        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

1/1 [==============================] - 0s 43ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 1.6021358e-26, 0.0, 0.0, 2.3587592e-05, 5.5936773e-15, 1.606135e-22, 0.0, 0.0, 0.0, 0.0, 2.7970444e-38, 0.0, 1.3318425e-37, 0.99911076, 0.00086567015, 4.5296467e-34] 18
1/1 [==============================] - 0s 14ms/step
[9.3968025e-30, 1.9466809e-19, 1.7943166e-35, 7.816009e-28, 4.098296e-27, 3.209263e-20, 4.4023437e-23, 2.6386684e-21, 0.86974823, 4.411929e-22, 1.1854897e-08, 0.0, 0.0, 1.0411191e-30, 6.7044776e-24, 1.12345886e-20, 1.0413344e-06, 6.867439e-20, 0.1302507, 1.673526e-14, 1.8258265e-16] 8
1/1 [==============================] - 0s 14ms/step
[0.0, 9.478624e-24, 2.2143892e-37, 8.33565e-38, 2.3813983e-31, 1.4236125e-35, 1.903531e-33, 9.0669685e-35, 0.9319116, 0.0, 1.7133003e-25, 0.0, 0.0, 0.0, 6.4160107e-37, 1.3268758e-29, 8.00467e-24, 1.6936429e-33, 0.06808833, 6.453018e-17, 2.9127283e-28] 8
1/1 [==============================] - 0s 14ms/step
[4.04997e-33, 1.786535e-32, 2.3928183e-33, 0.0, 1.2163345

1/1 [==============================] - 0s 13ms/step
[9.309912e-32, 1.6521135e-12, 8.372379e-12, 2.429454e-37, 2.7676075e-09, 6.8189265e-30, 4.835139e-23, 4.1474185e-24, 0.0008996709, 2.5416225e-28, 4.5968205e-31, 0.0, 6.200448e-36, 0.0, 3.7692175e-23, 0.9990711, 1.0507154e-21, 1.5910871e-16, 3.1854135e-08, 2.9155703e-05, 6.1856373e-28] 15
1/1 [==============================] - 0s 13ms/step
[8.732675e-31, 4.132953e-13, 2.2578985e-14, 0.0, 1.355375e-13, 1.0114955e-28, 5.2101576e-26, 6.488646e-27, 1.0, 5.801861e-30, 8.9653e-33, 2.1979832e-38, 1.5623279e-33, 0.0, 5.9511435e-25, 4.133742e-09, 1.2736379e-10, 7.1267385e-19, 5.930839e-13, 2.5186966e-09, 3.6532062e-29] 8
1/1 [==============================] - 0s 13ms/step
[2.3781537e-27, 3.630361e-09, 0.0007778758, 1.4705089e-33, 2.7980943e-06, 2.5629611e-23, 8.0355023e-26, 1.7624242e-22, 0.01018398, 1.3680872e-29, 6.166835e-28, 7.287326e-34, 4.195458e-24, 0.0, 1.7033947e-18, 0.9852116, 9.66327e-13, 1.160777e-18, 6.804811e-09, 0.0038236645, 2.3

1/1 [==============================] - 0s 13ms/step
[8.083248e-34, 2.7305752e-27, 1.6979154e-12, 1.2353779e-32, 2.3630712e-19, 5.33191e-34, 9.5177e-29, 2.9692961e-27, 0.9995937, 0.0, 2.4754997e-38, 0.0, 2.3968276e-25, 0.0, 5.6513626e-31, 1.2894374e-14, 6.183224e-23, 7.809827e-24, 0.0004063567, 1.7810659e-21, 1.3315306e-30] 8
1/1 [==============================] - 0s 13ms/step
[5.892962e-38, 1.3088121e-31, 2.4888643e-11, 0.0, 9.102324e-26, 0.0, 0.0, 5.5863413e-34, 0.9999974, 0.0, 0.0, 0.0, 2.639688e-26, 0.0, 0.0, 4.755902e-19, 6.1547963e-22, 1.991691e-32, 2.6106634e-06, 2.540461e-19, 3.4895086e-30] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 3.170541e-20, 0.048184317, 9.462351e-34, 1.8368638e-15, 8.71992e-34, 1.8206963e-34, 3.862761e-26, 0.9515321, 0.0, 0.0, 0.0, 3.0947223e-27, 0.0, 9.736811e-38, 6.672847e-05, 6.208254e-21, 1.2403408e-30, 0.00021688215, 7.01401e-18, 6.8666245e-34] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 1.5978407e-33, 2.0071372e-21, 0

1/1 [==============================] - 0s 14ms/step
[0.0, 5.8038853e-34, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.3768846e-22, 0.0, 2.835743e-32, 1.1001852e-29, 1.2567127e-17, 0.0, 4.4726918e-36, 2.1648005e-36, 1.7186236e-36, 0.0, 5.8618513e-33, 6.08321e-26, 0.0] 2
1/1 [==============================] - 0s 14ms/step
[0.0, 0.0, 1.942377e-10, 0.0, 0.0, 1.633404e-36, 0.0, 0.0, 1.0, 0.0, 1.4771982e-17, 0.0, 0.0, 0.0, 4.9557436e-16, 4.8092317e-38, 4.5558616e-15, 0.0, 1.7389733e-38, 2.6663163e-22, 0.0] 8
1/1 [==============================] - 0s 14ms/step
[0.0, 1.0987834e-26, 1.4858448e-05, 4.003812e-37, 0.0, 1.9409585e-24, 0.0, 2.5241672e-32, 0.99886525, 3.5110357e-36, 9.9002515e-11, 1.6812165e-37, 1.8420479e-18, 1.6434923e-38, 1.03573183e-22, 1.0138458e-35, 8.04528e-09, 1.398967e-35, 5.650937e-21, 0.0011199211, 2.4281881e-34] 8
1/1 [==============================] - 0s 14ms/step
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.1495835e-30, 0.0, 1.3085819e-22, 7.2756645e-30, 4.014918e-18, 0.0, 0.0, 0.0, 0

1/1 [==============================] - 0s 14ms/step
[2.4033715e-21, 4.4212126e-16, 1.4447642e-06, 2.370328e-22, 8.042683e-12, 3.0031126e-24, 3.8074738e-23, 1.1378846e-17, 1.505028e-05, 6.548647e-25, 2.678169e-29, 2.9683033e-26, 2.0629792e-21, 2.1217671e-30, 6.559511e-21, 1.5399063e-07, 4.3002154e-25, 8.8576e-17, 0.99998343, 3.2235645e-10, 6.965506e-22] 18
1/1 [==============================] - 0s 13ms/step
[4.746534e-10, 1.3345286e-13, 1.425752e-07, 4.2647824e-18, 4.3359663e-08, 4.559297e-16, 8.837819e-26, 7.131921e-13, 0.7673462, 3.6823798e-19, 6.98588e-18, 7.178176e-23, 4.1714798e-16, 1.0136199e-11, 1.0746091e-20, 6.074602e-11, 1.6057786e-23, 8.12326e-11, 0.23265347, 7.060697e-10, 1.4927795e-07] 8
1/1 [==============================] - 0s 13ms/step
[1.6161576e-20, 1.039474e-18, 2.5397622e-12, 2.6117392e-28, 4.2349913e-19, 1.2001412e-22, 8.525931e-32, 4.627902e-19, 0.025278188, 5.517018e-22, 1.7277712e-26, 2.7897711e-31, 7.0619314e-19, 8.0679405e-17, 1.4581122e-27, 7.97791e-15, 1.5162

...

1/1 [==============================] - 0s 13ms/step
[2.1530994e-31, 0.0, 7.469215e-38, 2.0147596e-28, 1.8396752e-26, 1.9968994e-22, 2.3564819e-05, 4.6281408e-32, 0.0058249617, 0.17140977, 3.2996522e-12, 0.0, 3.8342644e-22, 8.433848e-09, 2.4815071e-21, 2.8250968e-14, 1.696601e-13, 0.82273316, 8.612401e-06, 3.8415066e-13, 2.727114e-09] 17
1/1 [==============================] - 0s 13ms/step
[6.011616e-29, 1.0014962e-11, 2.1196178e-22, 1.6374914e-19, 4.6086613e-17, 5.7333404e-17, 2.0818897e-15, 8.415306e-26, 3.5500858e-10, 5.702317e-12, 3.1660506e-14, 0.0, 5.3674223e-21, 2.083041e-23, 1.401317e-13, 1.2819429e-18, 6.277631e-17, 0.9146282, 0.08537179, 3.832663e-13, 9.598298e-28] 17
1/1 [==============================] - 0s 13ms/step
[0.0, 1.437514e-26, 5.6807703e-27, 0.0, 0.0, 1.7587978e-15, 3.956065e-30, 0.0, 0.0035035105, 0.00021001486, 3.1588977e-18, 0.0, 5.6778923e-26, 0.0, 3.539809e-18, 1.4718431e-21, 0.9949602, 9.849901e-13, 0.001326287, 9.979901e-23, 4.6372444e-38] 16
1/1 [===========

1/1 [==============================] - 0s 13ms/step
[0.0, 2.2361703e-28, 2.3273691e-29, 6.430046e-36, 0.0, 3.4153933e-26, 0.0, 7.8659456e-36, 3.407833e-14, 5.294512e-13, 1.0, 0.0, 3.8835373e-38, 4.7908257e-32, 1.9783285e-20, 7.139858e-33, 1.9554115e-08, 9.291188e-27, 9.152882e-16, 1.0266312e-16, 1.18080656e-32] 10
1/1 [==============================] - 0s 13ms/step
[4.2631533e-36, 4.564491e-22, 2.3387444e-28, 2.3525492e-37, 0.0, 4.2507922e-30, 3.6444816e-35, 3.3210155e-32, 1.1097123e-12, 1.2302457e-12, 1.0, 0.0, 1.0776072e-34, 3.1637598e-28, 8.394087e-18, 2.5782313e-26, 1.0051024e-10, 1.4990044e-21, 3.223248e-18, 2.0141696e-22, 1.0708762e-31] 10
1/1 [==============================] - 0s 13ms/step
[5.580695e-38, 5.2297334e-16, 8.61317e-23, 0.0, 3.6730715e-30, 5.0715317e-34, 4.434706e-33, 7.414602e-30, 0.9741363, 1.15689964e-07, 2.1138742e-12, 0.0, 4.6805074e-30, 3.961884e-37, 1.4861907e-08, 1.5741203e-22, 0.025784966, 8.8579095e-13, 7.858085e-05, 4.2796773e-26, 0.0] 8
1/1 [=============

1/1 [==============================] - 0s 14ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.962861e-37] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.414014e-36] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7816679e-28, 0.0, 1.0, 0.0, 2.5290912e-38, 0.0, 0.0, 2.2106123e-31, 0.0, 1.3036406e-35, 0.0, 0.0, 2.399416e-22, 0.0, 0.0] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.642684e-35] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.685441e-37, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.

1/1 [==============================] - 0s 14ms/step
[8.0996235e-34, 4.4689768e-27, 1.3989083e-17, 5.2288942e-33, 5.8605786e-17, 7.638822e-33, 1.4628282e-21, 3.2424474e-19, 0.9999999, 6.391002e-33, 8.823384e-24, 0.0, 2.5363775e-29, 0.0, 3.3439626e-31, 5.6793767e-33, 9.540266e-08, 2.6610066e-26, 1.4309623e-18, 6.054646e-17, 1.7858367e-23] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 2.6460513e-37, 0.0, 0.0, 1.0, 0.0, 0.0, 6.9536855e-19, 1.2374778e-27, 1.2521834e-13, 0.0, 6.079545e-26, 4.8974024e-19, 0.0, 0.0, 1.0800504e-14, 0.0, 5.5045278e-36, 2.1847963e-34, 3.9293324e-37] 5
1/1 [==============================] - 0s 14ms/step
[0.0, 2.0839659e-36, 1.3292336e-31, 1.0942885e-35, 7.237276e-36, 1.167113e-23, 2.7096915e-28, 5.423004e-32, 1.0, 8.944285e-27, 5.3687816e-34, 0.0, 3.3222065e-32, 0.0, 0.0, 0.0, 9.1226445e-12, 0.0, 5.1848227e-17, 2.1316297e-20, 1.5598235e-22] 8
1/1 [==============================] - 0s 13ms/step
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0

1/1 [==============================] - 0s 14ms/step
[0.0, 2.2642657e-29, 1.3646695e-18, 1.5787032e-32, 0.0, 2.2391784e-08, 0.0, 4.437279e-21, 0.059464514, 0.3172144, 0.623321, 0.0, 8.5364035e-14, 8.709771e-21, 2.2749524e-34, 1.5224431e-36, 3.1945869e-15, 5.09651e-30, 4.5328294e-21, 4.597382e-22, 6.64194e-35] 10
1/1 [==============================] - 0s 14ms/step
[2.1206884e-35, 0.0, 2.3052363e-28, 0.0, 0.0, 0.0, 0.0, 1.7374822e-37, 1.0, 0.0, 0.0, 0.0, 1.9665104e-25, 0.0, 0.0, 0.0, 3.332914e-28, 0.0, 0.0, 0.0, 0.0] 8
1/1 [==============================] - 0s 14ms/step
[0.0, 0.0, 5.0170916e-06, 0.0, 0.0, 3.7579135e-14, 0.0, 4.146326e-28, 3.6995667e-11, 0.0021257272, 0.99786925, 0.0, 1.0926538e-18, 6.408406e-33, 0.0, 0.0, 2.1652997e-24, 0.0, 0.0, 3.4287785e-37, 5.366559e-37] 10
1/1 [==============================] - 0s 14ms/step
[0.0, 6.865063e-32, 3.5543786e-26, 0.0, 0.0, 0.000502172, 0.0, 1.0276059e-35, 0.0, 1.7149355e-17, 0.9994978, 2.699292e-23, 7.532283e-12, 2.7348685e-12, 0.0, 0.0, 